In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
tf.set_random_seed(1)
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
lr = 0.0001
training_iters = 100000
batch_size = 128
n_inputs = 28
n_steps = 28
n_hidden_units = 128
n_classes = 10

In [4]:
x = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.float32,[None,n_classes])
weights = {
    'in':tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    'out':tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}
biases = {
    'in':tf.Variable(tf.constant(0.1,dtype=tf.float32,shape=[n_hidden_units])),
    'out':tf.Variable(tf.constant(0.1,dtype=tf.float32,shape=[n_classes]))
}

In [5]:
def RNN(X,weights,biases):
    X = tf.reshape(X,[-1,n_inputs])
    X_in = tf.matmul(X,weights['in'])+biases['in']
    X_in = tf.reshape(X_in,[-1,n_steps,n_hidden_units])
    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units)
    init_state = cell.zero_state(batch_size,tf.float32)
    outputs,final_state = tf.nn.dynamic_rnn(cell,X_in,initial_state=init_state,time_major=False)   
    outputs = tf.unstack(tf.transpose(outputs,[1,0,2]))
    results = tf.matmul(outputs[-1],weights['out'])+biases['out']
    return results

In [6]:
pred = RNN(x,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))  
train_step = tf.train.AdamOptimizer(lr).minimize(cost)

In [7]:
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [13]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    step = 0
    while(step*batch_size<training_iters):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size,n_steps,n_inputs])
        sess.run(train_step,feed_dict={
            x:batch_xs,
            y:batch_ys,
        })
        if(step%20==0):
            print(sess.run(accuracy,feed_dict={
                x:batch_xs,
                y:batch_ys,
            }))
        step += 1

0.09375


0.304688


0.414062


0.554688


0.640625


...

0.914062


0.929688


0.890625


0.867188


0.890625
